# 实验2

In [5]:
import time
import random
import re
import warnings
import numpy as np

import jieba
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout, Conv1D, MaxPooling1D, Bidirectional

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
warnings.filterwarnings("ignore")

## 1. 数据预处理

### 1.1 加载label

In [10]:
labels = []
labels_list = []
with open('./label.txt', 'r', encoding='UTF-8') as f:
    for line in f:
        labels.append(eval(line))
        labels_list.extend(eval(line))

In [11]:
labels_list = dict(enumerate(set(labels_list)))
labels_list = {value:key for key,value in labels_list.items()}
labels_len = len(labels_list)
labels_len

233

总共233个类别

In [4]:
labels[:10]

[[],
 ['产品价格', '产品销量', '公司营销'],
 ['季度报告'],
 ['股票市场', 'A股'],
 ['IPO'],
 ['行业未来前景', '季度报告'],
 ['股票市场', 'A股'],
 ['公司业务数据', '业绩快报', '财务绩效'],
 ['政府征求意见'],
 []]

In [12]:
len(labels)

60158

### 1.2 加载title

In [7]:
titles = []
with open('./title.txt', 'r', encoding='UTF-8') as f:
    for line in f:
        titles.append(line.strip().strip('"'))

FileNotFoundError: [Errno 2] No such file or directory: './title.txt'

In [7]:
titles[:10]

['唯品会的特卖之道',
 '苏宁超市双十一暴涨287%，吃喝不愁更不在乎价格',
 '信而富去年亏损6658万美元，称正有序退出网贷业务',
 '快讯：得利斯跌停  报于9.55元',
 '三启赴港上市路 十二年“恩怨情仇” 阿里巴巴终圆梦香港',
 '新能源车产业链Q3盈利能力：动力电池最强 资源最弱',
 '国元证券：预估甬金股份发行价22.52元 对应市盈率16.24倍',
 '国信证券：腾讯控股新业务发展快速，广告下行',
 '图解“股票定向发行规则（征求意见稿）”',
 '曾鸣：创业的每一天，都是悬崖边的狂欢！']

In [8]:
len(titles)

60158

### 1.3 加载content

In [6]:
contents = []
with open('./content.txt', 'r', encoding='UTF-8') as f:
    for line in f:
        if line != "\n":
            contents.append(re.split(r'[\s, ######, 。, ！]',line.strip('"').strip()))

In [10]:
contents[2][:10]

['第一网贷讯',
 '11月13日，迟迟未公布2018年财报的信而富(XRF.N)发布2018年财报',
 '2018年，公司实现收入6661.9万美元，较2017年减少2110万美元；亏损为6558.5万美元，2017年亏损3647.9万元',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [11]:
len(contents)

60158

### 1.4 分词、去除停用词

In [12]:
line = "2019年1月3日 - python去除空格和换行符的方法 一、去除空格 strip().strip() # ...用replace('\',''),后边的串替换掉前边的posted @ 2016-07-18 08:53 ..."
pattern = re.compile(r'[^\u4e00-\u9fa5]')
chinese = re.sub(pattern, '', line)
chinese

'年月日去除空格和换行符的方法一去除空格用后边的串替换掉前边的'

In [13]:
length = len(labels)
print(length)
stopwords = [line.strip() for line in open('./stopwords.txt', 'r', encoding='utf-8').readlines()]
X = []
y = []
array_zero = [0 for index in range(length)]
pattern = re.compile(r'[^\u4e00-\u9fa5]')
Contents = []

60158


In [14]:
for content in contents:
    Content = []
    for c in content:
        c = re.sub(pattern, '', c)
        # print(c)
        if len(c) > 500:
            # print(len(c))
            c = c[:500]
        # 如果处理后为空 直接跳过
        if c == "":
            continue
        Content.append(c)
    Contents.append(Content)

In [15]:
Contents[30000][:5]

['产品整合年月出乎意料的是公司宣布将完全脱离磨床业务该项业务被出售给隶属于柯尔柏集团公司的斯来福林集团瓦尔特公司近年的漫长和高度成功的磨床制造历史宣告结束',
 '第二年随着推出采用涂层的',
 '老虎可转位刀片更有力的证明了企业的创新力再次为刀具寿命和生产效率设立了新标准',
 '公司步入世纪如果说百年的丰富经验主要强化了公司在高性能钻头领域的新优势那么则是为公司深化了研发上的特殊能力',
 '从此以后瓦尔特集团的这两个新成员分别以瓦尔特蒂泰克斯']

In [15]:
for i in range(length):
    if labels[i] == []:
        continue
    tmp = []
    for label in labels[i]:
        tmp.append(labels_list[label])
    y.append(tmp)
    
    # 分词
    cons = jieba.lcut(titles[i], cut_all=False)
    for c in Contents[i]:
        cons.extend(jieba.lcut(c, cut_all=False))
    x = []
    # 去除停用词
    for con in cons:
        if con not in stopwords:
            x.append(con)
    X.append(x)

IndexError: list index out of range

In [17]:
print(len(X),len(y))

54140 54140


In [18]:
f_X = open('./X.txt','w', encoding='utf-8')
for tmps in X:#ARTICLES
    for tmp in tmps:
        # print(tmp)
        f_X.write("{} ".format(tmp))
    f_X.write("\n")
f_X.close()

f_Y = open('./Y.txt','w', encoding='utf-8')
for tmps in y:#LABELS
    for tmp in tmps:
        # print(tmp)
        f_Y.write("{} ".format(tmp))
    f_Y.write("\n")
f_Y.close()

In [20]:
tmp_len = 0
for x in X:
    tmp_len = max(tmp_len, len(x))
tmp_len

12560